In [ ]:
import pulp
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
from pulp.apis import PULP_CBC_CMD
import pandas as pd
import tkinter as tk
from tkinter import ttk

num_bus = 8
days_of_schedule = range(7)
total_drivers = 8
num_short_shift_drivers = 4
num_long_shift_drivers = total_drivers - num_short_shift_drivers
hours_in_day = range(24)

problem = LpProblem(name="Составление расписания", sense=LpMinimize)

assign = LpVariable.dicts(
    "назначить", [(drv, coach, day, hour) for drv in range(total_drivers) 
                for coach in range(num_bus) for day in days_of_schedule for hour in hours_in_day],
    cat=LpBinary
)

for bus in range(num_bus):
    for day in days_of_schedule:
        for hour in hours_in_day:
            problem += lpSum(assign[drv, bus, day, hour] for drv in range(total_drivers)) <= 1

for drv in range(total_drivers):
    for day in days_of_schedule:
        shift_duration = 8 if drv < num_short_shift_drivers else 12
        for shift_start in range(0, 24 - shift_duration + 1):
            for bus in range(num_bus):
                if any(assign[drv, bus, day, hour].varValue == 1 for hour in range(shift_start, shift_start + shift_duration) if hour < 24):
                    for hour in range(shift_start, shift_start + shift_duration):
                        if hour < 24:
                            problem += assign[drv, bus, day, hour] == 1

for drv in range(total_drivers):
    for day in days_of_schedule:
        for bus in range(num_bus):
            for hour in hours_in_day[:-1]:
                problem += assign[drv, bus, day, hour + 1] >= assign[drv, bus, day, hour]

for bus in range(num_bus):
    for day in days_of_schedule:
        for hour in hours_in_day:
            problem += lpSum(assign[drv, bus, day, hour] for drv in range(total_drivers)) == 1

for bus in range(num_bus):
    for day in days_of_schedule:
        for hour in hours_in_day:
            problem += (
                lpSum(assign[drv, bus, day, hour] for drv in range(num_short_shift_drivers)) +
                lpSum(assign[drv, bus, day, hour] for drv in range(num_short_shift_drivers, total_drivers))
                <= 1
            )

for drv in range(num_short_shift_drivers):
    for day in days_of_schedule:
        if day < 5:
            for bus in range(num_bus):
                for hour in range(6, 14):
                    problem += assign[drv, bus, day, hour] == assign[drv, bus, 0, 6]
        else:
            problem += lpSum(assign[drv, coach, day, hour] for coach in range(num_bus) for hour in hours_in_day) == 0

for drv in range(num_short_shift_drivers, total_drivers):
    start_day = (drv - num_short_shift_drivers) % 3
    for day in days_of_schedule:
        if ((day - start_day) % 3 + 3) % 3 == 0:
            for bus in range(num_bus):
                problem += lpSum(assign[drv, bus, day, hour] for hour in range(14, 24)) == 10
                if day < 6:
                    problem += lpSum(assign[drv, bus, day + 1, hour] for hour in range(0, 12)) == 12
        else:
            problem += lpSum(assign[drv, coach, day, hour] for coach in range(num_bus) for hour in hours_in_day) == 0

for drv in range(total_drivers):
    for day in days_of_schedule:
        for hour in hours_in_day:
            problem += lpSum(assign[drv, coach, day, hour] for coach in range(num_bus)) <= 1

for drv in range(num_short_shift_drivers):
    problem += lpSum(assign[drv, coach, 0, 6] for coach in range(num_bus)) == 1

for bus in range(num_bus):
    for hour in range(6, 24):
        problem += lpSum(assign[drv, bus, 0, hour] for drv in range(total_drivers)) == 1

problem += lpSum(assign[drv, coach, day, hour] for drv in range(total_drivers) 
                 for coach in range(num_bus) for day in days_of_schedule for hour in hours_in_day)

status = problem.solve(pulp.PULP_CBC_CMD(msg=1))

print(f"Статус: {LpStatus[status]}")

shift_start_times = {"8ч": "06:00", "12ч": "06:00"}
shift_durations = {"8ч": 8, "12ч": 12}

days = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
driver_labels = [f"Водитель {drv + 1}" for drv in range(total_drivers)]

schedule_table = pd.DataFrame(index=driver_labels, columns=days)

for drv in range(total_drivers):
    shift_type = "8ч" if drv < num_short_shift_drivers else "12ч"
    duration = shift_durations[shift_type]
    for day in days_of_schedule:
        assigned_coaches = []
        start_times = []
        end_times = []
        for bus in range(num_bus):
            assigned_hours = [hour for hour in hours_in_day if assign[drv, bus, day, hour].varValue == 1]
            if assigned_hours:
                assigned_coaches.append(bus)
                start_time = min(assigned_hours)
                end_time = (start_time + duration) % 24
                start_times.append(f"{start_time:02d}:00")
                end_times.append(f"{end_time:02d}:00")
        if assigned_coaches:
            coach_assignments = ", ".join(map(str, assigned_coaches))
            shift_start = ", ".join(start_times)
            shift_end = ", ".join(end_times)
            schedule_table.iloc[drv, day] = f"Автобус {coach_assignments} ({shift_start} - {shift_end})"
        else:
            schedule_table.iloc[drv, day] = "Отдых"

shift_types = ["8ч" if drv < num_short_shift_drivers else "12ч" for drv in range(total_drivers)]
schedule_table.insert(0, "Продолжительность смены", shift_types)

def show_table_in_new_window(dataframe):
    window = tk.Tk()
    window.title("Расписание автобусов")
    window.geometry("1000x500")

    tree = ttk.Treeview(window, columns=list(dataframe.columns), show="headings", height=len(dataframe))

    for col in dataframe.columns:
        tree.heading(col, text=col, anchor=tk.CENTER)
        tree.column(col, anchor=tk.CENTER, width=130)

    for index, row in dataframe.iterrows():
        tree.insert("", "end", values=[index] + list(row))

    tree.pack(expand=True, fill="both", padx=10, pady=10)

    close_button = tk.Button(window, text="Закрыть", command=window.destroy, bg="#d9534f", fg="white", padx=10, pady=5)
    close_button.pack(pady=10)

    window.mainloop()

show_table_in_new_window(schedule_table)
